In [1]:
# scripts/ingest_and_transform.py

import pandas as pd

In [2]:
#Task 1: Load Data
# Load sales data
sales_df = pd.read_csv(r'C:\Users\DELL\Desktop\Expdia Data Engineer\Data\sales_data.csv')

# Load product master data
product_df = pd.read_csv(r'C:\Users\DELL\Desktop\Expdia Data Engineer\Data\product_master.csv')

In [3]:
# Print first few rows to confirm
print("Sales Data Sample:\n", sales_df.head())
print("\nProduct Master Data Sample:\n", product_df.head())

Sales Data Sample:
   transaction_id store_id product_id  quantity_sold  sale_amount  \
0         TXN001     S001       P001              2          400   
1         TXN002     S002       P003              1          200   

  transaction_date  
0       2024-12-01  
1       2024-12-01  

Product Master Data Sample:
   product_id     category
0       P001  Electronics
1       P002      Fashion


In [4]:
# Task 2: Data Cleaning & Transformation
# Step 1: To clean the data(removes rows with any missing (NaN) values)
sales_df.dropna(inplace = True)

# Step 2: Filter out rows where sale_amount <= 0
sales_df = sales_df[sales_df['sale_amount']  >  0]

In [5]:
# Step 3: convert 'transaction_date' to datetime
sales_df['transaction_date'] = pd.to_datetime(sales_df['transaction_date'])

# Step 3: add day_of _week column from the transaction_date
sales_df['day_of_week'] = sales_df['transaction_date'].dt.day_name()

In [6]:
# Print few rows to confirm
print("Sales Data:\n", sales_df.head())

Sales Data:
   transaction_id store_id product_id  quantity_sold  sale_amount  \
0         TXN001     S001       P001              2          400   
1         TXN002     S002       P003              1          200   

  transaction_date day_of_week  
0       2024-12-01      Sunday  
1       2024-12-01      Sunday  


In [7]:
# Step 4: To calculate revenue_per_unit 
sales_df['revenue_per_unit'] = sales_df['sale_amount'] / sales_df['quantity_sold'] 

# Print few rows to confirm
print("Sales Data:\n", sales_df.head())

Sales Data:
   transaction_id store_id product_id  quantity_sold  sale_amount  \
0         TXN001     S001       P001              2          400   
1         TXN002     S002       P003              1          200   

  transaction_date day_of_week  revenue_per_unit  
0       2024-12-01      Sunday             200.0  
1       2024-12-01      Sunday             200.0  


In [8]:
# Task 3: Generate Summary Reports
# Step 1: Total sales and quantity sold grouped by store and product
sales_summary = sales_df.groupby(['store_id', 'product_id']).agg(
    total_sales = ('sale_amount', 'sum'),
    total_quantity_sold = ('quantity_sold', 'sum')
).reset_index()

#print few rows to confirm 
print(sales_summary.head())

  store_id product_id  total_sales  total_quantity_sold
0     S001       P001          400                    2
1     S002       P003          200                    1


In [9]:
# Step 2: Top 5 products by total revenue
top_5_product = sales_df.groupby('product_id')['sale_amount'].sum().nlargest(5).reset_index()
top_5_product.rename(columns={'sale_amount':'total_revenue'}, inplace = True)

#print few rows to confirm 
print(top_5_product.head())

  product_id  total_revenue
0       P001            400
1       P003            200


In [10]:
# Step 3: The day of the week with the highest overall sales
sales_by_day = sales_df.groupby('day_of_week')['sale_amount'].sum().reset_index()

# sort to find the top day 
top_day = sales_by_day.sort_values(by = 'sale_amount', ascending = False).head(1)

# print the top day
print(top_day)

  day_of_week  sale_amount
0      Sunday          600


In [11]:
sales_summary.to_csv('sales_summary.csv', index=False)
top_5_product.to_csv('top_5_products.csv', index=False)
sales_by_day.to_csv('sales_by_day.csv', index=False)